In [1]:
import sqlalchemy
from bs4 import BeautifulSoup
import requests
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_date
from pyspark.sql.functions import *
import pandas as pd
from pyspark.sql.types import *
from sqlalchemy import create_engine
import os
import json
import time

In [2]:
# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Soccer Data") \
    .getOrCreate()

In [42]:
# def fetch_matchday(self):
log_name = "matchday"
url = f"https://www.sofascore.com/api/v1/sport/football/scheduled-events/2024-05-09"
response = requests.get(url)
status_code = response.status_code

if status_code != 200:
    print(f"Error code: {status_code}. Stop jobs!")
else:
    match_in_day = response.json()
    match_in_day = match_in_day['events']

    # Normalize the nested JSON data
    df_matchday = pd.json_normalize(match_in_day)
    df_matchday.rename(columns=lambda col: col.replace('.', '_'), inplace=True)
    df_matchday.rename(columns={"id": "match_id"}, inplace=True)
    df = df_matchday[df_matchday['tournament_category_sport_name'] == 'Football']
    print("Dataframe is here!!!")
    print(df)
    list_match_id = df["match_id"].astype(str).tolist()
    # self.export(log_name, df)
    # Fetch and export shotmap data
    # shotmap_df = shotmap(list_match_id)
    # if shotmap_df is not None:
    #     self.export("shotmap", shotmap_df)

# def shotmap(self, list_match_id):
#     shotmap_data = []
# 
#     for match_id in list_match_id:
#         url = f"https://www.sofascore.com/api/v1/event/{match_id}/shotmap"
#         try:
#             response = requests.get(url)
#             status_code = response.status_code
#             if status_code == 200:
#                 data = response.json()
#                 for event in data:
#                     event['match_id'] = match_id  # Add match_id to each event
#                 shotmap_data.extend(data)
#             else:
#                 print(f"Failed to fetch data for match ID {match_id}. Status code: {status_code}")
#         except Exception as e:
#             print(f"An error occurred for match ID {match_id}: {e}")
#         time.sleep(1)
# 
#     if shotmap_data:
#         # Normalize the nested JSON data
#         df_shotmap = pd.json_normalize(shotmap_data)
#         df_shotmap.rename(columns=lambda col: col.replace('.', '_'), inplace=True)
#         return df_shotmap
#     else:
#         return None

Dataframe is here!!!
     customId  winnerCode  aggregatedWinnerCode  hasGlobalHighlights  \
0     xdbsEdb         1.0                   1.0                 True   
1      QHwLdb         1.0                   1.0                False   
2     Qdbsceb         3.0                   1.0                 True   
3     TdbsIhb         3.0                   2.0                 True   
4       PsVbb         1.0                   1.0                False   
..        ...         ...                   ...                  ...   
173  jPisvNDb         1.0                   NaN                False   
174  MucsBjFb         2.0                   NaN                 True   
175   Sknsvmn         1.0                   NaN                 True   
176   DmnsEmn         1.0                   NaN                False   
177  FmnsvRRb         2.0                   NaN                 True   

    hasEventPlayerStatistics hasEventPlayerHeatMap  detailId  \
0                      False                 False

In [9]:
print(list_match_id)

['12173508', '12173471', '12173472', '12173517', '12173516', '12172425', '12172419', '12172525', '12172526', '12172437', '12172434', '12172441', '12172453', '12172509', '12172501', '12172345', '12172348', '12172359', '12172356', '12172350', '12172339', '12172333', '12172328', '12172329', '12214127', '12213731', '12310661', '12310660', '12310659', '12310668', '12255885', '12255883', '12255882', '12255880', '12146236', '11457108', '12310473', '12060302', '12060303', '12060304', '12287635', '12097656', '12097663', '12097660', '12097664', '12004537', '11453188', '11453182', '11453194', '11453178', '12295020', '12295017', '12295018', '12295013', '12091457', '11378835', '12026342', '12026337', '12026344', '11424125', '11451575', '11451642', '11451574', '11451572', '11451573', '11451571', '12277222', '12277221', '12297504', '12297506', '12297496', '12297499', '12288965', '12288963', '12288962', '12288968', '12288964', '12288971', '12162795', '12162793', '12115569', '12264393', '12270392', '12

In [51]:
# list_match_id = ["12173508"]
match_statistics_data = []

# for match_id in list_match_id:
for match_id in list_match_id:
    url = f"https://www.sofascore.com/api/v1/event/{match_id}/statistics"
    try:
        response = requests.get(url)
        status_code = response.status_code
        if status_code == 200:
            data = response.json()
            # Assuming 'statistics' is a list containing match statistics
            for event in data['statistics']:
                for group in event['groups']:
                    for stat_item in group['statisticsItems']:
                        # Append extracted data to the match_statistics_data list
                        match_statistics_data.append({
                            'period': event['period'],
                            'groupName': group['groupName'],
                            'name': stat_item['name'],
                            'home': stat_item['home'],
                            'away': stat_item['away'],
                            'compareCode': stat_item['compareCode'],
                            'statisticsType': stat_item['statisticsType'],
                            'valueType': stat_item['valueType'],
                            'homeValue': stat_item['homeValue'],
                            'awayValue': stat_item['awayValue'],
                            'renderType': stat_item['renderType'],
                            'key': stat_item['key']
                        })
            df_match_statistics = pd.json_normalize(match_statistics_data)
            df_match_statistics['match_id'] = match_id
        else:
            # Print an error message if the request fails
            print(f"Failed to fetch data for match ID {match_id}. Status code: {status_code}")
    except requests.exceptions.RequestException as e:
        # Handle specific exceptions related to requests
        print(f"An error occurred for match ID {match_id}: {e}")

# Convert match statistics data to a DataFrame if data was fetched successfully
# if df_match_statistics:
#     df_match_statistics.rename(columns=lambda col: col.replace('.', '_'), inplace=True)
    
    print(df_match_statistics)
else:
    print("No data fetched.")
#                     match_lineups_data.append(player_data)
#         else:
#             print(f"Failed to fetch data for match ID {match_id}. Status code: {status_code}")
#     except Exception as e:
#         print(f"An error occurred for match ID {match_id}: {e}")
#     time.sleep(1)
# 
# if match_lineups_data:
#     # Normalize the nested JSON data
#     df_match_lineups = pd.json_normalize(
#         match_lineups_data,
#         sep='_',
#         record_path=['statistics'],
#         meta=['team_type', 'match_id', ['player', 'name'], ['player', 'slug'], ['player', 'shortName'],
#               ['player', 'position'], ['player', 'jerseyNumber'], ['player', 'userCount'], ['player', 'id'],
#               ['player', 'country', 'alpha2'], ['player', 'country', 'alpha3'], ['player', 'country', 'name'],
#               ['player', 'marketValueCurrency'], ['player', 'dateOfBirthTimestamp']]
#     )
# 
#     df_match_lineups.rename(columns=lambda col: col.replace('.', '_'), inplace=True)
#     print(df_match_lineups.head())  # Print first few rows of DataFrame for verification
# else:
#     print("No match lineups data available.")

    period       groupName              name  home  away  compareCode  \
0      ALL  Match overview   Ball possession   57%   43%            1   
1      ALL  Match overview    Expected goals  2.95  0.35            1   
2      ALL  Match overview       Big chances     5     0            1   
3      ALL  Match overview       Total shots    19     8            1   
4      ALL  Match overview  Goalkeeper saves     4     5            2   
..     ...             ...               ...   ...   ...          ...   
114    2ND       Defending     Interceptions     5     4            1   
115    2ND       Defending        Recoveries    21    20            1   
116    2ND       Defending        Clearances     4    16            2   
117    2ND     Goalkeeping       Total saves     2     3            2   
118    2ND     Goalkeeping        Goal kicks     4     9            2   

    statisticsType valueType  homeValue  awayValue  renderType  \
0         positive     event       42.0       20.0       

In [52]:
df_match_statistics.columns

Index(['period', 'groupName', 'name', 'home', 'away', 'compareCode',
       'statisticsType', 'valueType', 'homeValue', 'awayValue', 'renderType',
       'key', 'match_id'],
      dtype='object')

In [47]:
match_id = "12173508"
url = f"https://www.sofascore.com/api/v1/event/{match_id}/statistics"
try:
    response = requests.get(url)
    status_code = response.status_code
    if status_code == 200:
        data = response.json()
        # Assuming 'statistics' is a list containing match statistics
        for event in data['statistics']:
            for group in event['groups']:
                for stat_item in group['statisticsItems']:
                    # Append extracted data to the match_statistics_data list
                    match_statistics_data.append({
                        'period': event['period'],
                        'groupName': group['groupName'],
                        'name': stat_item['name'],
                        'home': stat_item['home'],
                        'away': stat_item['away'],
                        'compareCode': stat_item['compareCode'],
                        'statisticsType': stat_item['statisticsType'],
                        'valueType': stat_item['valueType'],
                        'homeValue': stat_item['homeValue'],
                        'awayValue': stat_item['awayValue'],
                        'renderType': stat_item['renderType'],
                        'key': stat_item['key']
                    })
    else:
        # Print an error message if the request fails
        print(f"Failed to fetch data for match ID {match_id}. Status code: {status_code}")
except requests.exceptions.RequestException as e:
    # Handle specific exceptions related to requests
    print(f"An error occurred for match ID {match_id}: {e}")

In [48]:
response = requests.get(url)
status_code = response.status_code
print(status_code)

200


In [19]:
df=pd.json_normalize(data)
print(df)

   confirmed                                       home.players  \
0       True  [{'player': {'name': 'Daniel Carvajal', 'slug'...   

  home.supportStaff home.formation home.playerColor.primary  \
0                []          4-4-2                   ffffff   

  home.playerColor.number home.playerColor.outline  \
0                  000000                   ffffff   

  home.playerColor.fancyNumber home.goalkeeperColor.primary  \
0                       222226                       4be438   

  home.goalkeeperColor.number  ... away.formation away.playerColor.primary  \
0                      ffffff  ...        4-2-3-1                   090606   

  away.playerColor.number away.playerColor.outline  \
0                  8d287e                   090606   

  away.playerColor.fancyNumber away.goalkeeperColor.primary  \
0                       ffffff                       f92b91   

  away.goalkeeperColor.number away.goalkeeperColor.outline  \
0                      cff149                  

In [40]:
url = "https://www.sofascore.com/newcastle-united-manchester-united/DYBe#12240571"
response = requests.get(url)